## A stateless oracle (3): keeping the oracle up to date
#### 09.3 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-02-15
* Part 3: The transactions that keep the oracle up to date
* Parts 1-4 are only relevant if you want to **create** an Oracle
* Only parts 5-6 are needed to **use** the oracle.

**Note** that these transactions will typically run in regular intervals (every 5 min) on a Linux server, using the `cron` service of Linux. You can, however, run the manually to see what happens.

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [4]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Load additional oracle accounts
cred_oracle = load_credentials('credentials_oracle')
Price = cred_oracle['Price']
Reserve = cred_oracle['Reserve']
oracle_id = cred_oracle['oracle_id']

In [36]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import Multisig
from algosdk.future.transaction import PaymentTxn, MultisigTransaction
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig

import algosdk.error
import json
import base64
import hashlib

In [37]:
from pyteal import *

In [38]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [39]:
import json
import requests
import pandas as pd
import numpy as np
import time

In [64]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

#### Get information about the oracle coin

In [5]:
print('https://testnet.algoexplorer.io/asset/{}'.format(oracle_id))

https://testnet.algoexplorer.io/asset/77534697


## Transfer coins as a function of price and holdings
* This is the code that needs to be deployed on the remote server

In [60]:
# get current price
price_info = cg.get_price(ids='algorand', vs_currencies='usd')
usdalgo = price_info['algorand']['usd']
print(usdalgo)

0.741683


In [61]:
# get current holdings
holdings_Price = asset_holdings(algod_client, Price['public'])
oracle_Price = [holding['amount'] for holding in holdings_Price if holding['unit']=='USDALGO'][0]
oracle_Price = int(1e6*oracle_Price)
holdings_Reserve = asset_holdings(algod_client, Reserve['public'])
oracle_Reserve = [holding['amount'] for holding in holdings_Reserve if holding['unit']=='USDALGO'][0]
oracle_Reserve = int(1e6*oracle_Reserve)

print(usdalgo)
print(oracle_Price)
print(oracle_Reserve)

0.741683
742146
999257854


In [63]:
holdings_oracle = int(usdalgo*1e6)         # this is how many coins Price *should* hold

# make transfers
if holdings_oracle != oracle_Price:
    # A transaction is needed
    if holdings_oracle > oracle_Price:
        # Price does not have enough coins
        # Reserve needs to transfer to Price
        amt = int(holdings_oracle-oracle_Price)
        sender = Reserve
        receiver = Price
    else:
        # Price has too many coins
        # Price needs to transfer to Reserve
        amt = int(oracle_Price-holdings_oracle)
        sender = Price
        receiver = Reserve

    # === transfer TXN (must be a multisig!!) ===
    # Step 1: prepare
    sp = algod_client.suggested_params()          
    txn = AssetTransferTxn(
        sender = sender['public'],                
        sp=sp,
        receiver=receiver['public'],              
        amt=amt,                                  
        index=oracle_id                          
        )                               

    # Step 2+3: Sign + send
    stxn = txn.sign(sender['private'])
    txid = algod_client.send_transaction(stxn)

    # Step 4: Wait for confirmation
    txinfo = wait_for_confirmation(algod_client, txid)

Current round is  20463830.
Waiting for round 20463830 to finish.
Waiting for round 20463831 to finish.
Transaction 4RZB4EM5LSPPJVGAB2DZIPXYZ5Z2V2HGJGMATILDRSBRFSCBYU3Q confirmed in round 20463832.
